First connect notebook to google drive and load data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import os
import seaborn as sns; sns.set()
import numpy as np

In [ ]:
import pickle
import pandas as pd

df = pd.read_csv("data/emotions_analysis_adjusted_intensity.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)

In [ ]:
df['intensity'] = df['adjusted_intensity']
df.drop(['adjusted_intensity'], axis=1, inplace=True)

In [ ]:
df.head()

,created_at,tweet_id,tweet,likes,retweet_count,user_screen_name,user_join_date,user_followers_count,state,president_hashtag,dominant_party,others,joy,sadness,anger,surprise,disgust,fear,intensity,dominant_emotion
0,2020-10-25 17:03:17,4044763136,President @realDonaldTrump holds three rallies...,5782,1215,toddstarnes,2008-07-21 14:09:17,149808,Tennessee,joebiden,republican,0.022981,0.001447,0.001967,0.005207,0.002169,0.962983,0.003247,-4.828442,disgust
1,2020-10-23 02:20:41,2425868288,Proud Boys hearing #Biden call them the Poor B...,3357,370,search4swag,2014-04-04 19:49:10,15086,Missouri,joebiden,republican,0.874201,0.111448,0.005809,0.001819,0.002023,0.003363,0.001336,1.082761,others
2,2020-11-03 05:10:23,3902234624,"Dixville Notch, NH just announced their return...",2448,345,ChrisDJackson,2009-02-11 23:50:33,22857,Tennessee,joebiden,republican,0.963535,0.025358,0.002936,0.001687,0.002452,0.003087,0.000946,0.226769,others
3,2020-11-04 07:53:37,4241973248,I won! I am Tennessee State Representative Tor...,2035,204,TorreyHarris901,2020-04-12 09:52:01,1508,Tennessee,joebiden,republican,0.867814,0.120836,0.001326,0.003704,0.002075,0.002078,0.002167,1.178119,others
4,2020-10-29 23:55:32,3377831936,Follow the Ukraine cookie crumbs #Biden #Biden...,1643,705,GrrrGraphics,2010-03-30 17:37:29,228780,Montana,joebiden,republican,0.070463,0.002036,0.002695,0.008080,0.001362,0.913128,0.002236,-4.588379,disgust


Then group tweets by state and sum tweets intensity

In [ ]:
grouped_states_emotions_intensity = pd.DataFrame(df.groupby("state")["intensity"].apply(sum))

In [ ]:
grouped_states_emotions_intensity


,intensity
state,
Alabama,-789.800892
Alaska,-337.272234
Arizona,-137.855021
Arkansas,-566.636397
California,-848.531926
Colorado,9.659295
Connecticut,-7.261259
Delaware,32.187193
Florida,-1643.665769


ten group by dominant party in each state

In [ ]:
grouped_party_dominance_intensity = pd.DataFrame(df.groupby("dominant_party")["intensity"].apply(np.mean))

In [ ]:
grouped_party_dominance_intensity

,intensity
dominant_party,
democrat,-0.607186
republican,-1.249697
swing,-0.713950


Then use spacy to tokenize and lemmatize

In [ ]:
import spacy 
nlp = spacy.load("en_core_web_sm")

texts = df.tweet

processed_texts = [text for text in nlp.pipe(texts, 
                                              disable=["ner",
                                                       "parser"])]

df['processed_texts'] = processed_texts

In [ ]:
df_list = df.groupby(["dominant_party", "dominant_emotion"])["processed_texts"].apply(list)

In [ ]:
names = ['democrat_anger', 'democrat_disgust', 'democrat_fear', 'democrat_joy', 'democrat_others', 'democrat_sadness', 'democrat_surprise',
         'republican_anger', 'republican_disgust', 'republican_fear', 'republican_joy', 'republican_others', 'republican_sadness', 'republican_surprise',
         'swing_anger', 'swing_disgust', 'swing_fear', 'swing_joy', 'swing_others', 'swing_sadness', 'swing_surprise']

In [ ]:
print(names[0])
df_list[0][0:5]

democrat_anger


[Why can’t @NBCNews choose a professional, seasoned moderator so the president can answer questions without her interrupting him every moment. The moderator is supposed to be neutral. Her hostility is so evident. #SavannahGuthrie #Trump,
 A man who has refused to release his taxes to the public has the brass balls to claim that #JoeBiden will raise your taxes!  Joe will raise taxes, but only in corporations, who currently pay ZERO and BILLIONAIRES.  Don’t forget asshole #Trump paid....$750.00 in taxes last year.,
 Why would any thinking American ever vote for hideous #DonaldTrump? He’s killed 220,000 million of us!  He wants to take healthcare away from 30 million people DURING A FUCKING PANDEMIC!  Who does that?  And why?  Because people’s lives aren’t terrible enough, they must be worse?,
 THIS is UNCONSCIONABLE. #Trump pulled out of the scheduled townhall. #Biden KEPT HIS COMMITMENT &amp; HIS WORD and moved forward. Now @nbc @nbcnews is going to REWARD 45 for breaking his commitment

In [ ]:
df_list

dominant_party  dominant_emotion
democrat        anger               [(Why, ca, n’t, @NBCNews, choose, a, professio...
                disgust             [(#, trump, wants, to, throw, out, military, b...
                fear                [(Are, you, there, ,, God, ?, It, 's, me, ,, B...
                joy                 [(Tonight, a, woman, in, the, audience, of, Tr...
                others              [(America, Assembled, !, 🇺, 🇸, 🎉, \n\n, @JoeBi...
                sadness             [(Wonderful, speeches, tonight, by, President,...
                surprise            [(OMG, !,  , #, JoeBiden, speaks, like, a, hum...
republican      anger               [(You, Got, ta, Love, Our, President, !, \n, V...
                disgust             [(President, @realDonaldTrump, holds, three, r...
                fear                [(Do, n't, be, misled, by, #, Trump, :, The, 3...
                joy                 [(@BarackObama, was, the, first, President, I,...
                other

In [ ]:
def tokenize_verb_noun_adj(processed_texts_input):
  tokenized_texts_out = [[word.lemma_ for word in processed_text if (word.pos_ == 'VERB' or word.pos_ == 'NOUN' or word.pos_ == 'ADJ') 
                                and not word.is_stop and not word.is_punct] for processed_text in processed_texts_input]
  return tokenized_texts_out

from collections import Counter

flatten = lambda t: [item for sublist in t for item in sublist]

In [ ]:
tokenized_flat_texts = []
for i in range(len(df_list)):
  processed_texts = df_list[i]
  tokenized_text = tokenize_verb_noun_adj(processed_texts)
  flatten_text = flatten(tokenized_text)
  tokenized_flat_texts.append(flatten_text)



In [ ]:
tokenized_flat_texts[1][0:10]

['trump',
 'want',
 'throw',
 'military',
 'ballot',
 'bone',
 'spur',
 'People',
 'get',
 'strand']

Then get distinctive words for each emotion

In [ ]:
from scipy.stats import chi2_contingency
def distinctive_words(target_corpus, reference_corpus):
    counts_c1 = Counter(target_corpus) # don't forget to flatten your texts!
    counts_c2 = Counter(reference_corpus)
    vocabulary = set(list(counts_c1.keys()) + list(counts_c2.keys()))
    freq_c1_total = sum(counts_c1.values()) 
    freq_c2_total = sum(counts_c2.values()) 
    results = []
    for word in vocabulary:
        freq_c1 = counts_c1[word]
        freq_c2 = counts_c2[word]
        freq_c1_other = freq_c1_total - freq_c1
        freq_c2_other = freq_c2_total - freq_c2
        llr, p_value,_,_ = chi2_contingency([[freq_c1, freq_c2], 
                      [freq_c1_other, freq_c2_other]],
                      lambda_='log-likelihood') 
        if freq_c2 / freq_c2_other > freq_c1 / freq_c1_other:
            llr = -llr
        result = {'word':word, 
                    'llr':llr,
                    'p_value': p_value}
        results.append(result)
    results_df = pd.DataFrame(results)
    return results_df

ANGER

In [ ]:
print(names[0], names[7], names[14])

democrat_anger republican_anger swing_anger


In [ ]:
#most distinctive in democrat anger comparing to the other anger
results_df_anger = distinctive_words(tokenized_flat_texts[0], (tokenized_flat_texts[7] + tokenized_flat_texts[14]))
results_df_anger.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
1432,goprussia,31.020941,2.552595e-08
2482,workload,26.329566,2.878483e-07
2735,@morningmika,26.329566,2.878483e-07
4375,able,24.146064,8.929849e-07
5339,in,21.743114,3.117066e-06
2413,reduce,21.743114,3.117066e-06
2255,kag,20.901831,4.834322e-06
104,@gopleader,17.034388,3.670893e-05
651,t,15.578149,7.916419e-05
1569,hope,15.468580,8.388820e-05


In [ ]:
#most distinctive in republican anger comparing to the other anger
results_df_anger = distinctive_words(tokenized_flat_texts[7], (tokenized_flat_texts[0] + tokenized_flat_texts[14]))
results_df_anger.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
2964,horse,27.229001,1.807274e-07
2598,limit,26.905560,2.136434e-07
2913,mendacity,26.905560,2.136434e-07
4765,quintessence,26.905560,2.136434e-07
5184,apprehension,26.905560,2.136434e-07
2932,faculty,26.905560,2.136434e-07
2464,toddler,26.905560,2.136434e-07
5322,form,26.741997,2.325127e-07
3365,reprehensible,23.282938,1.398350e-06
593,ignoble,16.101379,6.004029e-05


In [ ]:
#most distinctive in swing anger comparing to the other anger
results_df_anger = distinctive_words(tokenized_flat_texts[14], (tokenized_flat_texts[0] + tokenized_flat_texts[7]))
results_df_anger.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
1386,flip,12.618366,0.000382
2838,ONEV1,11.879205,0.000568
4587,tired,10.434768,0.001237
1662,😡,10.434768,0.001237
808,yrs,9.843913,0.001704
608,blue,8.269728,0.004031
291,stimulus,7.686289,0.005564
2991,today,7.101598,0.007702
1579,real,6.937185,0.008442
372,‍,6.751035,0.009369


DISGUST

In [ ]:
print(names[1], names[8], names[15])

democrat_disgust republican_disgust swing_disgust


In [ ]:
#most disgust in democrat comparing to the other disgust
results_df_disgust = distinctive_words(tokenized_flat_texts[1], (tokenized_flat_texts[8] + tokenized_flat_texts[15]))
results_df_disgust.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
8645,debt,53.652375,2.392942e-13
1479,t,46.032384,1.163144e-11
3387,grow,33.038160,9.036771e-09
6647,skyrocket,28.960738,7.386026e-08
8198,wokeaf,24.559970,7.203267e-07
5583,recession,24.162906,8.852094e-07
10213,dustbin,20.184309,7.032744e-06
8277,trumpslide,20.184309,7.032744e-06
9584,epic,16.106491,5.987847e-05
1166,🚫,15.983834,6.388570e-05


In [ ]:
#most disgust in republican comparing to the other disgust
results_df_disgust = distinctive_words(tokenized_flat_texts[8], (tokenized_flat_texts[1] + tokenized_flat_texts[15]))
results_df_disgust.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
1721,trumplie,79.909926,3.918730e-19
3161,DitchMoscowMitch,74.798909,5.211849e-18
10629,republicansforbiden,63.032395,2.033346e-15
4094,trumpisalaughingstock,57.741468,2.989339e-14
5163,@gop,54.229377,1.783996e-13
1647,cabal,52.726544,3.833747e-13
5453,cockroach,50.802401,1.021487e-12
8137,subpoena,47.975153,4.316548e-12
6121,censor,44.942172,2.029399e-11
3279,trumpispathetic,39.010114,4.216156e-10


In [ ]:
#most disgust in swing comparing to the other disgust
results_df_disgust = distinctive_words(tokenized_flat_texts[15], (tokenized_flat_texts[8] + tokenized_flat_texts[1]))
results_df_disgust.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
6077,wtpsenate,36.320653,1.673807e-09
2640,votebluenomatterwho,34.063239,5.334969e-09
4569,wtpblue,29.559735,5.421965e-08
5511,wtpbiden,20.615341,5.614442e-06
1867,trumpthepeoplespresident,20.615341,5.614442e-06
6991,medium,19.765525,8.754764e-06
7828,resist,17.380297,3.059815e-05
9392,wakeup,17.305693,3.182321e-05
5268,🏽,17.246179,3.283573e-05
11274,maga2020,17.117362,3.513960e-05


FEAR

In [ ]:
print(names[2], names[9], names[16])

democrat_fear republican_fear swing_fear


In [ ]:
#FEAR in democrat comparing to the other fear
results_df_fear = distinctive_words(tokenized_flat_texts[2], (tokenized_flat_texts[9] + tokenized_flat_texts[16]))
results_df_fear.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
689,night,7.731440,0.005427
430,electoral,5.833292,0.015726
619,believe,5.159131,0.023124
1032,democracy,4.214424,0.040082
1410,await,4.157830,0.041442
271,put,4.157830,0.041442
815,kill,4.157830,0.041442
294,winter,4.157830,0.041442
1139,term,3.306325,0.069014
1137,nation,3.306325,0.069014


In [ ]:
#FEAR in republicans comparing to the other fear
results_df_fear = distinctive_words(tokenized_flat_texts[9], (tokenized_flat_texts[2] + tokenized_flat_texts[16]))
results_df_fear.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
210,trump,14.445074,0.000144
1210,virus,11.665824,0.000637
965,estimate,9.732579,0.001810
104,year,7.261027,0.007047
1022,community,6.345582,0.011767
161,afraid,6.093641,0.013567
64,economy,5.838793,0.015677
1029,people,5.528448,0.018710
1236,die,4.800269,0.028455
935,@gop,4.072420,0.043589


In [ ]:
#FEAR in swing comparing to the other fear
results_df_fear = distinctive_words(tokenized_flat_texts[16], (tokenized_flat_texts[2] + tokenized_flat_texts[9]))
results_df_fear.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
462,scary,15.546047,0.000081
852,increase,14.940834,0.000111
339,detail,12.895111,0.000329
649,impeach,12.895111,0.000329
796,hunterbidenemail,12.895111,0.000329
1037,likelihood,12.895111,0.000329
480,damn,12.895111,0.000329
794,president,12.062097,0.000515
1229,ballot,12.062097,0.000515
1255,😱,11.174742,0.000829


JOY

In [ ]:
print(names[3], names[10], names[17])

democrat_joy republican_joy swing_joy


In [ ]:
#joy in democrat comparing to the other fear
results_df_joy = distinctive_words(tokenized_flat_texts[3], (tokenized_flat_texts[10] + tokenized_flat_texts[17]))
results_df_joy.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
3024,tip,14.937238,0.000111
4147,track,13.118834,0.000292
1097,show,13.081954,0.000298
2071,celebrate,11.559773,0.000674
4159,✨,8.531032,0.003491
2927,cut,8.531032,0.003491
3439,1st,8.341409,0.003875
2836,celebration,8.063015,0.004518
2822,honest,7.703269,0.005512
3803,remain,7.587461,0.005878


In [ ]:
#joy in republican comparing to the other fear
results_df_joy = distinctive_words(tokenized_flat_texts[10], (tokenized_flat_texts[3] + tokenized_flat_texts[17]))
results_df_joy.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
292,😭,14.140290,0.000170
1231,dog,14.104417,0.000173
523,🇬,11.232751,0.000804
1840,list,11.232751,0.000804
3238,❤,9.872904,0.001677
726,trump,9.847138,0.001701
466,open,8.266932,0.004037
1550,keepamericagreat,8.033244,0.004593
2406,result,7.915266,0.004902
1771,process,6.438368,0.011168


In [ ]:
#joy in swing comparing to the other fear
results_df_joy = distinctive_words(tokenized_flat_texts[17], (tokenized_flat_texts[3] + tokenized_flat_texts[10]))
results_df_joy.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
1762,sign,12.480038,0.000411
4107,rally,10.099506,0.001483
1743,conservative,9.844123,0.001704
4344,question,8.755070,0.003087
4013,face,5.579250,0.018175
3089,proud,5.270103,0.021695
2887,maga2020,5.050470,0.024619
4377,thing,5.018201,0.025082
1221,meet,4.970807,0.025779
1133,soulofanation,4.773841,0.028895


Other

In [ ]:
print(names[4], names[11], names[18])

democrat_others republican_others swing_others


In [ ]:
#other in democrat comparing to the other fear
results_df_other = distinctive_words(tokenized_flat_texts[4], (tokenized_flat_texts[11] + tokenized_flat_texts[18]))
results_df_other.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
6604,daily,76.995163,1.713772e-18
15135,approval,75.193982,4.266656e-18
14083,able,68.743090,1.121632e-16
15522,video,56.061282,7.024696e-14
6002,@libertynation,50.281639,1.331899e-12
11943,https://t.co/mazbcwoimy,48.323868,3.613274e-12
10192,sponsor,46.861854,7.617097e-12
13711,rating,45.483956,1.538943e-11
17471,in,45.243539,1.739936e-11
8868,await,42.246243,8.047444e-11


In [ ]:
#other in republican comparing to the other fear
results_df_other = distinctive_words(tokenized_flat_texts[11], (tokenized_flat_texts[14] + tokenized_flat_texts[18]))
results_df_other.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
6843,MAGA2020LANDSLIDE,60.929461,5.915692e-15
13603,➖,59.156285,1.456335e-14
546,trump,53.465837,2.631291e-13
4344,magaa,48.074054,4.104216e-12
4705,pip,41.478734,1.191601e-10
9439,biden,29.843879,4.682730e-08
11922,makeamericagreatagain,27.426833,1.631516e-07
6851,trend,22.874129,1.729643e-06
9190,debate2020,22.566690,2.029735e-06
1528,trumpsawhinybitch,17.061439,3.618973e-05


In [ ]:
#other in swing comparing to the other fear
results_df_other = distinctive_words(tokenized_flat_texts[18], (tokenized_flat_texts[11] + tokenized_flat_texts[4]))
results_df_other.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
8772,wtpsenate,192.647091,8.405214e-44
6534,wtpblue,188.465297,6.875915e-43
837,por,146.990121,7.886618e-34
7933,wtpbiden,89.638444,2.859140e-21
11292,resist,58.272545,2.282067e-14
6094,voteblue,57.590019,3.228597e-14
7120,🏻,53.138785,3.107948e-13
12919,eleccione,49.509530,1.974104e-12
16335,demvoice1,48.835856,2.783052e-12
3049,life,36.261974,1.724969e-09


SADNESS

In [ ]:
print(names[5], names[12], names[19])

democrat_sadness republican_sadness swing_sadness


In [ ]:
#sadness in democrat comparing to the other fear
results_df_sadness = distinctive_words(tokenized_flat_texts[5], (tokenized_flat_texts[12] + tokenized_flat_texts[19]))
results_df_sadness.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
287,million,7.651397,0.005673
1134,minute,5.910788,0.015048
184,tear,5.853206,0.015549
374,beloved,4.216621,0.040030
1227,visit,4.216621,0.040030
463,move,4.216621,0.040030
1161,heartbroken,3.372212,0.066305
1040,nation,3.072449,0.079629
863,hear,3.009031,0.082802
410,grave,2.597552,0.107029


In [ ]:
#sadness in republican comparing to the other fear
results_df_sadness = distinctive_words(tokenized_flat_texts[12], (tokenized_flat_texts[5] + tokenized_flat_texts[19]))
results_df_sadness.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
909,😘,7.493306,0.006193
661,debate,7.493306,0.006193
1078,matter,4.754978,0.029214
11,break,4.630117,0.031415
1056,sad,3.927152,0.047512
300,make,3.507299,0.061099
1038,beat,3.431547,0.063962
494,sign,3.431547,0.063962
1072,month,3.431547,0.063962
543,bury,3.431547,0.063962


In [ ]:
#sadness in swing comparing to the other fear
results_df_sadness = distinctive_words(tokenized_flat_texts[19], (tokenized_flat_texts[12] + tokenized_flat_texts[5]))
results_df_sadness.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
601,pray,5.578726,0.018180
655,love,5.490554,0.019119
339,well,5.046946,0.024669
413,get,4.527308,0.033358
213,hope,3.610066,0.057431
1045,look,3.459075,0.062906
1101,support,3.459075,0.062906
19,tonight,2.982183,0.084186
512,leader,2.982183,0.084186
613,way,2.654428,0.103262


SUPRISE

In [ ]:
print(names[6], names[13], names[20])

democrat_surprise republican_surprise swing_surprise


In [ ]:
#suprise in democrat comparing to the other fear
results_df_suprise = distinctive_words(tokenized_flat_texts[6], (tokenized_flat_texts[13] + tokenized_flat_texts[20]))
results_df_suprise.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
432,project,6.020324,0.014142
454,rate,5.071497,0.024322
620,surpass,5.071497,0.024322
581,additional,5.071497,0.024322
459,gained,5.071497,0.024322
497,official,5.071497,0.024322
555,go,3.836997,0.050133
251,BIDEN,3.185632,0.074288
633,exciting,3.185632,0.074288
355,word,3.185632,0.074288


In [ ]:
#suprise in republican comparing to the other fear
results_df_suprise = distinctive_words(tokenized_flat_texts[13], (tokenized_flat_texts[6] + tokenized_flat_texts[20]))
results_df_suprise.sort_values('llr', ascending=False).head(20)

,word,llr,p_value
502,day,5.381865,0.020347
485,lol,4.343842,0.037143
201,come,2.373299,0.123426
47,tonight,2.373299,0.123426
258,projection,2.096964,0.147592
132,elect,2.096964,0.147592
181,unbelievable,2.096964,0.147592
856,predict,2.096964,0.147592
88,complete,2.096964,0.147592
13,fire,2.096964,0.147592


In [ ]:
#suprise in swing comparing to the other fear
results_df_suprise = distinctive_words(tokenized_flat_texts[20], (tokenized_flat_texts[6] + tokenized_flat_texts[13]))
results_df_suprise.sort_values('llr', ascending=False).head(20)


,word,llr,p_value
755,award,7.025287,0.008037
816,feeling,3.830262,0.050335
456,😱,3.830262,0.050335
457,think,2.355392,0.124850
128,election2020,2.332110,0.126730
310,ad,2.332110,0.126730
558,hope,2.332110,0.126730
571,leader,2.332110,0.126730
766,count,1.773164,0.182991
846,thing,1.773164,0.182991
